In [1]:
import hashlib
import random

In [2]:
import string
import json

In [3]:
import binascii

In [4]:
import numpy as np
import pandas as pd
import pylab as pl

In [5]:
import logging
import datetime
import collections

In [6]:
import Crypto
import Crypto.Random

In [7]:
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

In [8]:
class Client:
    def __init__(self):
        random = Crypto.Random.new().read
        self._private_key = RSA.generate(1024, random)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

In [9]:
Nishtha = Client()
print (Nishtha.identity)

30819f300d06092a864886f70d010101050003818d0030818902818100b4c37f27f0c3c3b2ff7c4c3ea7a210e7302435f61d9db7092a07a97bb34d24456bbdc12e40bf1153ccde64d55bf8c94a3332df14b3f2127633e31d9e1f114df1109daa3fd2930abcedf9b748b69230bc05dcec142611723cb1d6ee749052f30a0857cd05f51a9eaca9c2dbf52051dee7946e1ad52722ec0b20bb97527d8eab190203010001


In [10]:
class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()
        
    def to_dict(self):
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity

        return collections.OrderedDict({'sender': identity,'recipient': self.recipient,'value': self.value,'time' : self.time})
    
    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')
    

In [11]:
Nishtha = Client()
Sakshi = Client()
Tanishq= Client()
Harshit = Client()
Selena=Client()
Elizabeth=Client()

In [12]:
t = Transaction(Nishtha,Sakshi.identity,5.0)
t2 = Transaction(Nishtha,Tanishq.identity,5.0)
signature = t.sign_transaction()
print (signature)

2d86d76a55a501359242ddbd5f47c5a49058377e06681f4ab9b06b9df2ef51f554f08a16182fd0db9c80d16f8d8d2674e8c13d413a7382b2f00d3b505ce50b22daca5861159aaee90138e1cad06b02b0013bb78b54367d6ec64e670d5ebe098c02e4b2feb4c8de50d0f719d51ad8c8d1d2c9c5e816f78b53c1475104429680b9


In [13]:
def display_transaction(transaction):
    dict = transaction.to_dict()
    print ("sender: " + dict['sender'])
    print ('-----')
    print ("recipient: " + dict['recipient'])
    print ('-----')
    print ("value: " + str(dict['value']))
    print ('-----')
    print ("time: " + str(dict['time']))
    print ('-----')

In [14]:
transactions = []

In [15]:
t1 = Transaction( Nishtha,Sakshi.identity, 15.0)
t1.sign_transaction()
transactions.append(t1)

In [16]:
t2 = Transaction( Nishtha, Tanishq.identity, 6.0)
t2.sign_transaction()
transactions.append(t2)
t3 = Transaction( Tanishq, Sakshi.identity, 2.0)
t3.sign_transaction()
transactions.append(t3)
t4 = Transaction( Tanishq,Nishtha.identity, 4.0)
t4.sign_transaction()
transactions.append(t4)
t5 = Transaction(Nishtha, Sakshi.identity, 7.0)
t5.sign_transaction()
transactions.append(t5)
t6 = Transaction( Nishtha, Harshit.identity, 3.0)
t6.sign_transaction()
transactions.append(t6)
t7 = Transaction(Harshit, Selena.identity, 8.0)
t7.sign_transaction()
transactions.append(t7)
t8 = Transaction(Elizabeth,Selena.identity, 1.0)
t8.sign_transaction()
transactions.append(t8)
t9 = Transaction(Selena,Harshit.identity, 5.0)
t9.sign_transaction()
transactions.append(t9)
t10 = Transaction(Nishtha,   Elizabeth.identity, 3.0)
t10.sign_transaction()
transactions.append(t10)

In [17]:
for transaction in transactions:
    display_transaction (transaction)
    print ('--------------')


sender: 30819f300d06092a864886f70d010101050003818d0030818902818100e6bc2540132ba2b41c3d6a291b595fbafd6b24f6fec52f539f09ecfc37fc0f59e3bcfd718d2bacc1db502b85da95a718417739617df0a4387b7e9ffea7130947c53e24acb2581e6fb5f9e6593e7e4bc381239192249b6aefe86d49a1fcaf321d962676c5891d613d5bf72961728e2503ddb8b0c903760c5a831fa69f4beeb7890203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100b36aa58206e09699dc29068d951bc2f5988d9f8066f1de044d1af332d9f35b7079143eb6f3e95c59361a96dad49edba1409aaef6177f3b3d2acc8c8a715e75ddc601e582940a0b57e0b3eae82992d89d0798a31e6520e891fedb9505138cb850425163c1d2eb33f02c0cf376f5d27314b0d3f3eba05f989bdcd93b5443ce79cf0203010001
-----
value: 15.0
-----
time: 2021-02-10 16:36:40.888915
-----
--------------
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100e6bc2540132ba2b41c3d6a291b595fbafd6b24f6fec52f539f09ecfc37fc0f59e3bcfd718d2bacc1db502b85da95a718417739617df0a4387b7e9ffea7130947c53e24acb2581e6fb5f9e6593e7e4bc381239192249b6aefe86d4

In [18]:
class Block:
    last_block_hash = ""
    def __init__(self):
        self.verified_transactions = []
        self.previous_block_hash = ""
        self.Nonce = ""

In [19]:
Nishtha=Client()

In [20]:
t0 = Transaction ( "Genesis", Nishtha.identity, 500.0)

In [21]:
block0 = Block()

In [22]:
block0.previous_block_hash = None
Nonce = None

In [23]:
block0.verified_transactions.append (t0)

In [24]:
digest = hash (block0)
last_block_hash = digest

In [25]:
SATANCoins = []

In [26]:
def dump_blockchain (self):
    print ("Number of blocks in the chain: " + str(len (self)))
    for x in range (len(SATANCoins)):
        block_temp = SATANCoins[x]
        print ("block # " + str(x))
        for transaction in block_temp.verified_transactions:
            display_transaction (transaction)
            print ('--------------')
    print ('=====================================')

In [27]:
SATANCoins.append (block0)

In [28]:
dump_blockchain(SATANCoins)

Number of blocks in the chain: 1
block # 0
sender: Genesis
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100e72d292d67072e22bc193af929a0033b0f23eb311b1a9625635db7cd83befb44c7b8b1776a45ef1b5c421a220351b47bf6b8313058757398e9626bf2f1c3b1af3d3724dbca2fb4ae5dd28360c66a9c8a74ae7be2f43be0e3a6e0be0a6871c27c0e0647e830faa758c34c87dffe72e251fcfce22da880e571f8e52cccdd052ae30203010001
-----
value: 500.0
-----
time: 2021-02-10 16:36:42.452387
-----
--------------


In [29]:
def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

In [30]:
def mine(message, difficulty=1):
    assert difficulty >= 1
    prefix = '1' * difficulty
    for i in range(1000):
        digest = sha256(str(hash(message)) + str(i))
        if digest.startswith(prefix):
            print ("after " + str(i) + " iterations found nonce: "+ digest)
            return digest

In [31]:
mine ("test message", 2)

after 418 iterations found nonce: 11eadc91897fbb190ed0c933e6d0d1966fd94a7b85013117f3abd622c1995f2b


'11eadc91897fbb190ed0c933e6d0d1966fd94a7b85013117f3abd622c1995f2b'

In [32]:
last_transaction_index = 0

In [33]:
block = Block()

In [34]:
for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1

In [35]:
#Nishtha adds the block
block.previous_block_hash = last_block_hash
block.Nonce = mine (block, 2)
digest = hash (block)
SATANCoins.append (block)
last_block_hash = digest

after 32 iterations found nonce: 112bbc7ea7b261aaede346667147dea930a6ffb7e358f676152400a02d80bfe4


In [36]:
# Now Sakshi adds a block
block = Block()
for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1

In [37]:
block.previous_block_hash = last_block_hash
block.Nonce = mine (block, 2)
digest = hash (block)
SATANCoins.append (block)
last_block_hash = digest

after 173 iterations found nonce: 117b88cdf90f3ba4f55c3316a7317c24f34bc77e642d8518e65fe0c82dd1bea1


In [38]:
#Now say Tanishq adds a block
block = Block()
for i in range(3):
    temp_transaction = transactions[last_transaction_index]
    display_transaction (temp_transaction)
    block.verified_transactions.append (temp_transaction)
    last_transaction_index += 1

sender: 30819f300d06092a864886f70d010101050003818d0030818902818100ad5e2a9f0fc8178b5b0db232374a6d9ed1165ea441a6cf39667eb2aae77116dbcbbd080b3dbd77eec5d411c884400653b6cdbb2023e7146852efe3653f52e35b539a93436822b6e81e2cc1ea7ce78111c03ced7015f26f16439d2ef8aae69717d872744eb64de346620b5a744db22939a7f22f48d4e052884e10ff0155b83ec90203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d00308189028181009c25cc73c60423bd346ef3e5e3827d1be17651481e44350216471f1296c875c9d2d64e0cce0a6b7d0ec7bb322296741050b92f661ae32bf3bd1a59281bc695a37809836b6e0109a2cbf66fb88b095d90b472ba0a2696b7f0e192b6ff3c5bf1b2a28bb8d7e25c1497be670f9c98e6e7c3b04170122b7a6169d57da58977daf1c90203010001
-----
value: 8.0
-----
time: 2021-02-10 16:36:41.133898
-----
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100c5395b48968850f48a9e736655b1011775821b16dd5022b831e7236c2206477e13eb9341a2930389030598b031ec9731f1bd3b58b7ccc44ae6ee10b22eb0ad30b2fc3ae8d00b27e250b53d34ad2a88641198255e2f725bdfaa16b2e093fde1bd52c53

In [39]:
block.previous_block_hash = last_block_hash
block.Nonce = mine (block, 2)
digest = hash (block)
SATANCoins.append (block)
last_block_hash = digest

after 140 iterations found nonce: 11e4db3db6d649b4a5264c57e4914e3a77e6dce92e56c2c09174226e0b1f77f1


In [40]:
dump_blockchain(SATANCoins)

Number of blocks in the chain: 4
block # 0
sender: Genesis
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100e72d292d67072e22bc193af929a0033b0f23eb311b1a9625635db7cd83befb44c7b8b1776a45ef1b5c421a220351b47bf6b8313058757398e9626bf2f1c3b1af3d3724dbca2fb4ae5dd28360c66a9c8a74ae7be2f43be0e3a6e0be0a6871c27c0e0647e830faa758c34c87dffe72e251fcfce22da880e571f8e52cccdd052ae30203010001
-----
value: 500.0
-----
time: 2021-02-10 16:36:42.452387
-----
--------------
block # 1
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100e6bc2540132ba2b41c3d6a291b595fbafd6b24f6fec52f539f09ecfc37fc0f59e3bcfd718d2bacc1db502b85da95a718417739617df0a4387b7e9ffea7130947c53e24acb2581e6fb5f9e6593e7e4bc381239192249b6aefe86d49a1fcaf321d962676c5891d613d5bf72961728e2503ddb8b0c903760c5a831fa69f4beeb7890203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100b36aa58206e09699dc29068d951bc2f5988d9f8066f1de044d1af332d9f35b7079143eb6f3e95c59361a96dad49edba1409aae